# Kubernetes

In [1]:
#!pip install ipynb
from ipynb.fs.full.facturxWidget import *

from ipynb.fs.full.kubernetesTool import *

from ipynb.fs.full.invoiceSmartcontract import *

from ipynb.fs.full.blockchain import *

In [3]:
_blockchain = Blockchain(channel_name='invoicing26',shared_fs='10.15.130.111:80',sharepoint=SharePoint())
cc = Chaincode()

In [ ]:
cc.__dict__

In [4]:
_blockchain.addPeer('aubonmoulin', hostname='fr',ca_country='FR',ca_province='Provence-Alpes-Cote d\'Azur',ca_locality='Malaucene')
_blockchain.addPeer('lyon-impots', hostname='gouv.fr',ca_country='FR',ca_province='Auvergne-Rhone-Alpes',ca_locality='Lyon')
_blockchain.addPeer('austriafactor', hostname='at',ca_country='AT',ca_province='Steiermark',ca_locality='Graz')
_blockchain.addPeer('megustaolive', hostname='es',ca_country='ES',ca_province='Andalusia',ca_locality='Dos Hermanas')

### user

In [5]:
_blockchain.installWalletTool()

In [ ]:
_user = User(peer=_blockchain.peers['aubonmoulin'], name='Tony.Dubois@aubonmoulin.fr', channel_name=_blockchain.channel_name)
#User(peer=_blockchain.peers['aubonmoulin'], name='Alexandre.Payet@majolieboutique.net', channel_name=_blockchain.channel_name)

In [ ]:
_user.smart_contract.getAllRoles()

In [ ]:
_keys = _user.smart_contract.getAllKeys()[1]

In [ ]:
_user.smart_contract.deleteAsset(asset=_keys[7])

In [ ]:
for key,_peer in _blockchain.peers.items():
    _organizations = _peer.wallet_tool.getPeerOrganizations()
    for _organization in _organizations:
        porg = Organization(name=_organization)
        _users = _peer.wallet_tool.getPeerUsers(_organization)
        for _user in _users:
            puser = User(peer=_peer, name=_user, channel_name=_blockchain.channel_name)
            porg.addUser(user=puser)
        _peer.organizations[_organization]=porg
    

In [ ]:
_blockchain.peers['aubonmoulin'].organizations['aubonmoulin.fr'].users['Tony.Dubois@aubonmoulin.fr'].__dict__

#_blockchain.peers['lyon-impots'].organizations
#_blockchain.peers

In [ ]:
_smart_contract_aubonmoulin = InvoiceSmartContract(peer=_blockchain.peers['aubonmoulin'], user='Tony.Dubois@aubonmoulin.fr', channel_name=_blockchain.channel_name)

In [ ]:
_smart_contract_lyon_impots = InvoiceSmartContract(peer=_blockchain.peers['lyon-impots'], user='Auguste.Boucher@lyon-impots.gouv.fr', channel_name=_blockchain.channel_name)

In [ ]:
_smart_contract.getSubmittingClientIdentity()
_smart_contract.getAllRoles()

In [6]:
_blockchain.peers['aubonmoulin'].wallet_tool.addUser('Tony.Dubois','aubonmoulin.fr','aubonmoulin.fr')
_blockchain.peers['aubonmoulin'].wallet_tool.addUser('Stephanie.Hoarau','hotelsaintdenis.re','aubonmoulin.fr')
_blockchain.peers['aubonmoulin'].wallet_tool.addUser('Alexandre.Payet','majolieboutique.net','aubonmoulin.fr')
_blockchain.peers['austriafactor'].wallet_tool.addUser('Emma.Steiner','austriafactor.at','austriafactor.at')
_blockchain.peers['megustaolive'].wallet_tool.addUser('Pedro.Sanchez','megustaolive.es','megustaolive.es')
_blockchain.peers['lyon-impots'].wallet_tool.addUser('Auguste.Boucher','lyon-impots.gouv.fr','lyon-impots.gouv.fr')

('OK: 138 MiB in 84 packages\nOK: 138 MiB in 84 packages\ncreate_csr\nCreate csr  for Auguste.Boucher@lyon-impots.gouv.fr\nConfiguration\n  CA cert path:   ../ca.org1.example.com-cert.pem \n  CA key path:    ../priv_sk  \n  CN of new cert: Auguste.Boucher@lyon-impots.gouv.fr \n  Organisation:    \nOK: 138 MiB in 84 packages\nOK: 138 MiB in 84 packages\npush_csr\n10.15.130.111:80\n<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">\n<html><head>\n<title>405 Method Not Allowed</title>\n</head><body>\n<h1>Method Not Allowed</h1>\n<p>The requested method MKCOL is not allowed for the URL /csrs_to_sign/.</p>\n</body></html>\n<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">\n<html><head>\n<title>301 Moved Permanently</title>\n</head><body>\n<h1>Moved Permanently</h1>\n<p>The document has moved <a href="http://10.15.130.111/csrs_to_sign/Auguste.Boucher@lyon-impots.gouv.fr/">here</a>.</p>\n</body></html>\nReplace  http://10.15.130.111:80/csrs_to_sign/Auguste.Boucher@lyon-impots.gouv.fr/user.cs

In [ ]:
def appendRoles(peers):
    peer=peers['lyon-impots']
    appendRole(peer,user='Auguste.Boucher@lyon-impots.gouv.fr',role='TaxInspector')
    peer_taxinspector=peer
    #taxinspector=get_person_names_with_role('TaxInspector',roles)[0].split(',')[0].split('=')[1]  
    taxinspector='Auguste.Boucher@lyon-impots.gouv.fr'

    identity_taxinspector = getSubmittingClientIdentity(peer_taxinspector, user=taxinspector)[0]

    peer=peers['aubonmoulin']
    appendRole(peer,user='Tony.Dubois@aubonmoulin.fr',role='Seller',taxInspector=identity_taxinspector)
    #appendRole(peer,user='Stephanie.Hoarau',role='Buyer')
    #appendRole(peer,user='Alexandre.Payet',role='Buyer')

    peer=peers['austriafactor']
    appendRole(peer,user='Emma.Steiner@austriafactor.at',role='Factor',taxInspector=identity_taxinspector)

    peer=peers['megustaolive']
    appendRole(peer,user='Pedro.Sanchez@megustaolive.es',role='Buyer',taxInspector=identity_taxinspector)
    appendRole(peer,user='Pedro.Sanchez@megustaolive.es',role='Seller',taxInspector=identity_taxinspector)

In [ ]:
def getUser(_blockchain, _user):
    _org = _user.split('@')[1]
    for key,peer in _blockchain.peers.items():
        if _org in peer.organizations.keys():
            if _user in peer.organizations[_org].users.keys():
                return peer.organizations[_org].users[_user]

In [48]:
getUser(_blockchain,'Auguste.Boucher@lyon-impots.gouv.fr').smart_contract.appendRole(role='TaxInspector')
identity_taxinspector = getUser(_blockchain, 'Auguste.Boucher@lyon-impots.gouv.fr').smart_contract.getSubmittingClientIdentity()[0]
getUser(_blockchain,'Emma.Steiner@austriafactor.at').smart_contract.appendRole(role='Factor',taxInspector=identity_taxinspector)
getUser(_blockchain,'Tony.Dubois@aubonmoulin.fr').smart_contract.appendRole(role='Seller',taxInspector=identity_taxinspector)
getUser(_blockchain,'Pedro.Sanchez@megustaolive.es').smart_contract.appendRole(role='Seller',taxInspector=identity_taxinspector)
getUser(_blockchain,'Pedro.Sanchez@megustaolive.es').smart_contract.appendRole(role='Buyer',taxInspector=identity_taxinspector)

('', Output())

In [41]:
_keys = getUser(_blockchain,'Tony.Dubois@aubonmoulin.fr').smart_contract.getAllKeys()
_keys

(['asset14',
  'asset17',
  'asset3',
  'asset6',
  'roles_x509::CN=Auguste.Boucher@lyon-impots.gouv.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.lyon-impots.gouv.fr,O=lyon-impots.gouv.fr,L=Lyon,ST=Auvergne-Rhone-Alpes,C=FR',
  'roles_x509::CN=Emma.Steiner@austriafactor.at,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.austriafactor.at,O=austriafactor.at,L=Graz,ST=Steiermark,C=AT',
  'roles_x509::CN=Pedro.Sanchez@megustaolive.es,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.megustaolive.es,O=megustaolive.es,L=Dos,ST=Andalusia,C=ES',
  'roles_x509::CN=Tony.Dubois@aubonmoulin.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.aubonmoulin.fr,O=aubonmoulin.fr,L=Malaucene,ST=Provence-Alpes-Cote,C=FR'],
 Output())

In [37]:
_smart_contract = getUser(_blockchain,'Tony.Dubois@aubonmoulin.fr').smart_contract

In [47]:
_keys = getUser(_blockchain,'Tony.Dubois@aubonmoulin.fr').smart_contract.getAllKeys()[0]
_smart_contract = getUser(_blockchain,'Tony.Dubois@aubonmoulin.fr').smart_contract
_smart_contract.deleteAsset(asset=_keys[4])
_smart_contract.deleteAsset(asset=_keys[5])
_smart_contract.deleteAsset(asset=_keys[6])
_smart_contract.deleteAsset(asset=_keys[7])

('', Output())

In [49]:
_smart_contract.getAllRoles()

([{'name': 'x509::CN=Auguste.Boucher@lyon-impots.gouv.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.lyon-impots.gouv.fr,O=lyon-impots.gouv.fr,L=Lyon,ST=Auvergne-Rhone-Alpes,C=FR',
   'mspid': 'lyon-impotsMSP',
   'roles': ['TaxInspector'],
   'taxInspector': ''},
  {'name': 'x509::CN=Emma.Steiner@austriafactor.at,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.austriafactor.at,O=austriafactor.at,L=Graz,ST=Steiermark,C=AT',
   'mspid': 'austriafactorMSP',
   'roles': ['Factor'],
   'taxInspector': 'x509::CN=Auguste.Boucher@lyon-impots.gouv.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.lyon-impots.gouv.fr,O=lyon-impots.gouv.fr,L=Lyon,ST=Auvergne-Rhone-Alpes,C=FR'},
  {'name': 'x509::CN=Pedro.Sanchez@megustaolive.es,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.megustaolive.es,O=megustaolive.es,L=Dos,ST=Andalusia,C=ES',
   'mspid': 'megustaoliveMSP',
   'roles': ['Seller', 'Buyer'],
   'taxInspector': 'x509::CN=Auguste.Boucher@lyon-impots.gouv.fr,OU

In [33]:
getUser(_blockchain,'Tony.Dubois@aubonmoulin.fr').smart_contract.getRoles()

({'name': 'x509::CN=Tony.Dubois@aubonmoulin.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.aubonmoulin.fr,O=aubonmoulin.fr,L=Malaucene,ST=Provence-Alpes-Cote,C=FR',
  'roles': ['aubonmoulinMSP'],
  'taxInspector': 'Seller'},
 Output())

In [ ]:
#_smart_contract_lyon_impots.getAllRoles()
_smart_contract_aubonmoulin.getRoles()
#_out
_smart_contract_lyon_impots.getRoles()
_smart_contract_lyon_impots.getAllKeys()

In [ ]:
_smart_contract_lyon_impots.getAllKeys()

## cleanup

In [ ]:
_blockchain.deletePeers()
#_blockchain.peers

## add channel

In [ ]:
_blockchain.add_channel('invoicing26')

## debug

In [ ]:
out2.outputs[1]['text'].splitlines()

In [ ]:
_out.outputs[0]['text'].splitlines()

## create peers

### endorse and join

In [ ]:
## endorse and join
import ipywidgets as widgets

#orderer_ip,orderer_hostname,orderer_port,channel_name,shared_fs_host

In [ ]:
_blockchain.endorse_and_join_all_peers()

## upload chaincode to sharepoint

In [ ]:
strlist = ['/bin/sh /opt/create-archive.sh',cc.hostname,cc.port,cc.label,cc.shared_fs]
cmd = toCmd(strlist)
print(cmd)
#cmd = export_cmd + '  && ' + cmd
output = executePeerCmd(getPod('cc-deployer'), cmd,cli='cc-deployer')

## install chaincode

In [66]:
_blockchain.installAndAproveChaincode( chain_code=cc)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   433  100   433    0     0  28866      0 --:--:-- --:--:-- --:--:-- 30928
Error: chaincode install failed with status: 500 - failed to invoke backing implementation of 'InstallChaincode': chaincode already successfully installed
2022-03-09 13:01:11.946 UTC [chaincodeCmd] ClientWait -> INFO 001 txid [76b4a3d5c0a7e3d9ab23e23f02c705daf805fc8c4604d59191a0c1a8d22fd483] committed with status (VALID) at peer0.aubonmoulin.fr:32432
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   433  100   433    0     0  54125      0 --:--:-- --:--:-- --:--:-- 54125
Error: chaincode install failed with status: 500 - failed to invoke backing implementation of 'InstallChaincode': chaincode already successfully installed
2022-03-09 13:01:1

In [ ]:
_blockchain.peers['aubonmoulin'].commitChaincode(_blockchain.channel_name,pkgid=cc.pkgid)

## add roles

In [75]:
deleteAllInvoices(User(peer=_blockchain.peers['aubonmoulin'], name='Tony.Dubois@aubonmoulin.fr', channel_name=_blockchain.channel_name).smart_contract)              

In [76]:
createInvoices(invoices,_blockchain)            

'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable


In [ ]:
from typing import Dict, Any
import hashlib
import json

def dict_hash(dictionary: Dict[str, Any]) -> str:
    """MD5 hash of a dictionary."""
    dhash = hashlib.md5()
    # We need to sort arguments so {'a': 1, 'b': 2} is
    # the same as {'b': 2, 'a': 1}
    encoded = json.dumps(dictionary, sort_keys=True).encode()
    dhash.update(encoded)
    return dhash.hexdigest()


In [18]:
getUser(_blockchain,get_CN_of_invoice(invoices.iloc[3],'sellerTradeParty')).__dict__

{'peer': <ipynb.fs.full.blockchain.KubernetesPeer at 0x7efeed68db20>,
 'name': 'Tony.Dubois@aubonmoulin.fr',
 'smart_contract': <ipynb.fs.full.invoiceSmartcontract.InvoiceSmartContract at 0x7efeed68d790>}

In [26]:
invoice = invoices.iloc[3]
seller = getUser(_blockchain,get_CN_of_invoice(invoice,'sellerTradeParty'))
buyer  = getUser(_blockchain,get_CN_of_invoice(invoice,'buyerTradeParty')) 
seller_id = seller.smart_contract.getSubmittingClientIdentity()[0]
buyer_id  = buyer.smart_contract.getSubmittingClientIdentity()[0]
asset     = seller.smart_contract.getEmptyInvoice()[0]
update_invoice(asset=asset,invoice=invoice,id=3,seller=seller_id,buyer=buyer_id)
seller.smart_contract.createInvoice(arglist=getArgList(asset))[1]

Output()

In [53]:
seller.smart_contract.getAllKeys()

(['asset14',
  'asset17',
  'asset3',
  'asset6',
  'roles_x509::CN=Auguste.Boucher@lyon-impots.gouv.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.lyon-impots.gouv.fr,O=lyon-impots.gouv.fr,L=Lyon,ST=Auvergne-Rhone-Alpes,C=FR',
  'roles_x509::CN=Emma.Steiner@austriafactor.at,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.austriafactor.at,O=austriafactor.at,L=Graz,ST=Steiermark,C=AT',
  'roles_x509::CN=Pedro.Sanchez@megustaolive.es,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.megustaolive.es,O=megustaolive.es,L=Dos,ST=Andalusia,C=ES',
  'roles_x509::CN=Tony.Dubois@aubonmoulin.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.aubonmoulin.fr,O=aubonmoulin.fr,L=Malaucene,ST=Provence-Alpes-Cote,C=FR'],
 Output())

In [52]:
seller.smart_contract.listInvoice(id='asset14')

({'ID': 'asset14',
  'owner': 'x509::CN=Tony.Dubois@aubonmoulin.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.aubonmoulin.fr,O=aubonmoulin.fr,L=Malaucene,ST=Provence-Alpes-Cote,C=FR',
  'buyer': 'x509::CN=Stephanie.Hoarau@hotelsaintdenis.re,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.aubonmoulin.fr,O=aubonmoulin.fr,L=Malaucene,ST=Provence-Alpes-Cote,C=FR',
  'hash': -2901837500345949450,
  'invoiceNumber': 'FA-2017-0009',
  'tax': 0,
  'netto': 530.75,
  'countryOrigin': 'FR',
  'countryBuyer': 'RE',
  'status': '',
  'received': False,
  'receivedOrder': False,
  'sold': False,
  'claimPaid': False,
  'claimPaidBy': '',
  'taxExemptionReason': 'Exoneration de TVA selon article 262 I du Code general des impots',
  'taxReceived': False},
 Output())

In [22]:
seller_id

'x509::CN=Tony.Dubois@aubonmoulin.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.aubonmoulin.fr,O=aubonmoulin.fr,L=Malaucene,ST=Provence-Alpes-Cote,C=FR'

In [ ]:
User(peer=_blockchain.peers['aubonmoulin'], name='Tony.Dubois@aubonmoulin.fr', channel_name=_blockchain.channel_name).smart_contract.listInvoices()

In [ ]:
invoice=invoices.iloc[3]
getUser(_blockchain,get_CN_of_invoice(invoice,'sellerTradeParty'))

In [ ]:
seller.smart_contract.getAllKeys()[0]

In [13]:
invoices = get_invoices()
#createInvoices(invoices,_blockchain.peers)
#invoices

In [ ]:
def get_org_of_invoice(invoice,party):
    _name=strip_accents(getPathValue(invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/' + party + '/definedTradeContact/')[0]['personName']['value'].replace(' ','.'))
    _email=getPathValue(invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/' + party + '/definedTradeContact/')[0]['emailURIUniversalCommunication']['uriid']['value']
    return _email.split('@')[1]

In [ ]:
User(peer=_blockchain.peers['aubonmoulin'], user='Tony.Dubois@aubonmoulin.fr', channel_name=_blockchain.channel_name).smart_contract.getSubmittingClientIdentity()

In [ ]:
get_CN_of_invoice(invoice,'sellerTradeParty')

In [ ]:
#list(asset)
_smart_contract_aubonmoulin.getEmptyInvoice()[0]

In [ ]:
InvoiceSmartContract(peer=_blockchain.peers['lyon-impots'], user='Auguste.Boucher@lyon-impots.gouv.fr').appendRole(role='TaxInspector')

In [ ]:
InvoiceSmartContract(peer=_blockchain.peers['aubonmoulin'], user='Tony.Dubois@aubonmoulin.fr').appendRole(role='Seller', taxInspector='')

In [ ]:
strlist = ['/bin/ls',  '/mnt/conf/organizations/peerOrganizations/']
peer=_blockchain.peers['aubonmoulin']
executePeerCmd(peer.pod_name,toCmd(strlist)).splitlines() 

In [ ]:
appendRoles(_blockchain.peers)
#peer=_blockchain.peers['lyon-impots']
#appendRole(peer,user='Auguste.Boucher@lyon-impots.gouv.fr',role='TaxInspector')

In [ ]:
peer=_blockchain.peers['lyon-impots']
getRoles(peer,user='Auguste.Boucher@lyon-impots.gouv.fr')[0]

In [ ]:
callopt='{"function":"AppendRole","Args":["x509::CN=Auguste.Boucher@lyon-impots.gouv.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.lyon-impots.gouv.fr,O=lyon-impots.gouv.fr,L=Lyon,ST=Auvergne-Rhone-Alpes,C=FR","TaxInspector",""]}'

In [ ]:
peer=_blockchain.peers['lyon-impots']
getAllKeys(peer)[1]

In [ ]:
peer=_blockchain.peers['lyon-impots']
callopt= '{"function":"GetAllKeys","Args":[' + ']}'
#peer.query(callopt,channel_name='invoicing18',user='Auguste.Boucher@lyon-impots.gouv.fr')
peer.query(callopt,channel_name='invoicing18',user='User1')

In [ ]:
getAllRoles(peer)
getRoles(peer, user='Auguste.Boucher@lyon-impots.gouv.fr')

In [ ]:
getRoleTransactions(peer)

In [ ]:
invoices = get_invoices()
createInvoices(invoices,_blockchain.peers)
#invoices

In [ ]:
peer=_blockchain.peers['aubonmoulin']
getPeerOrganizations(peer)

In [ ]:
get_peer_of_user('Tony.Dubois@aubonmoulin.fr',_blockchain.peers)

In [ ]:
peer=_blockchain.peers['aubonmoulin']
listInvoices(peer, user='Tony.Dubois@aubonmoulin.fr')[1]

In [ ]:
peer=_blockchain.peers['aubonmoulin']
InvoiceSmartContract(peer=peer, user='Tony.Dubois@aubonmoulin.fr').listInvoices()[0]

In [ ]:
InvoiceSmartContract(peer=peer, user='Tony.Dubois@aubonmoulin.fr').__dict__

In [ ]:
deleteAllInvoices(peer)  